In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('Collect Exploration Data') \
    .getOrCreate()

25/07/26 04:54:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
loans = spark.read.csv('archive/loan_2014_18.csv', header=True)

In [3]:
loans.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs:

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

display(loans.limit(5).toPandas())

25/07/26 04:54:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,0,120122535,12000.0,12000.0,12000.0,36 months,7.97%,375.88,A,A5,associate,10+ years,OWN,42000.0,Source Verified,2017-09-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=120122535,debt_consolidation,Debt consolidation,923xx,CA,27.74,0.0,Jun-1996,715.0,719.0,0.0,None,80.0,9.0,1.0,11457.0,37%,16.0,w,0.0,0.0,13500.44,13500.44,12000.0,1500.44,0.0,0.0,0.0,May-2020,2591.95,None,May-2020,694.0,690.0,0.0,None,1.0,Individual,None,None,None,0.0,0.0,30502.0,1.0,2.0,1.0,3.0,8.0,19045.0,73.0,2.0,4.0,7117.0,53.0,31000.0,1.0,1.0,2.0,7.0,3389.0,7144.0,53.9,0.0,0.0,131.0,255.0,1.0,1.0,0.0,14.0,None,8.0,None,0.0,2.0,6.0,2.0,2.0,7.0,7.0,9.0,6.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,57180.0,30502.0,15500.0,26180.0,None,None,None,None,None,None,None,None,None,None,None,None,N,None,None,None,None,None,None,None,None,None,None,None,None,None,None,N
1,1,119374887,32000.0,32000.0,32000.0,36 months,11.99%,1062.71,B,B5,Nurse,10+ years,MORTGAGE,155000.0,Source Verified,2017-09-01,Current,n,https://lendingclub.com/browse/loanDetail.action?loan_id=119374887,credit_card,Credit card refinancing,080xx,NJ,12.35,2.0,Sep-2005,715.0,719.0,1.0,10.0,None,20.0,0.0,48309.0,34.1%,42.0,w,6158.89,6158.89,31838.67,31838.67,25841.11,5997.56,0.0,0.0,0.0,May-2020,0.0,Jul-2020,May-2020,674.0,670.0,0.0,None,1.0,Individual,None,None,None,0.0,0.0,405751.0,2.0,1.0,1.0,1.0,8.0,15582.0,78.0,4.0,7.0,14049.0,40.0,142600.0,0.0,2.0,2.0,8.0,22542.0,81313.0,34.5,0.0,0.0,91.0,144.0,1.0,1.0,3.0,1.0,24.0,1.0,10.0,0.0,7.0,10.0,12.0,27.0,3.0,18.0,36.0,10.0,20.0,0.0,0.0,0.0,5.0,94.9,0.0,0.0,0.0,527034.0,63891.0,124200.0,20034.0,None,None,None,None,None,None,None,None,None,None,None,None,Y,CVD19SKIP,INCOMECURT,ACTIVE,2.0,0.0,Apr-2020,Jun-2020,Apr-2020,2.0,0.0,ACTIVE

## borrower


In [5]:
borrower_columns = ['id', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
     'verification_status', 'addr_state', 'zip_code', 'application_type']

borrower = loans.select(borrower_columns)
borrower.limit(10).toPandas()

,id,emp_title,emp_length,home_ownership,annual_inc,verification_status,addr_state,zip_code,application_type
0,120122535,associate,10+ years,OWN,42000.0,Source Verified,CA,923xx,Individual
1,119374887,Nurse,10+ years,MORTGAGE,155000.0,Source Verified,NJ,080xx,Individual
2,119321612,Driver,9 years,MORTGAGE,120000.0,Verified,TX,778xx,Individual
3,120122034,Senior Investigator,5 years,RENT,79077.0,Not Verified,VA,223xx,Individual
4,118659541,Registered Nurse,< 1 year,MORTGAGE,107000.0,Verified,TX,750xx,Joint App
5,119246018,CSM,10+ years,RENT,32000.0,Source Verified,IL,604xx,Individual
6,119183129,Therapist,3 years,MORTGAGE,55000.0,Not Verified,OR,971xx,Individual
7,120086024,Owner,< 1 year,MORTGAGE,120000.0,Not Verified,NY,136xx,Joint App
8,119417303,Area Director,5 years,RENT,90000.0,Not Verified,TX,799xx,Individual
9,120020939,pdr technician,10+ years,MORTGAGE,90000.0,Source Verified,MA,023xx,Individual


In [6]:
from pyspark.sql.functions import col, sum

borrower_nulls = borrower.select([sum(col(c).isNull().cast("int")).alias(c) for c in borrower.columns])
borrower_nulls.show()

[Stage 3:=================================>                        (7 + 5) / 12]

+---+---------+----------+--------------+----------+-------------------+----------+--------+----------------+
| id|emp_title|emp_length|home_ownership|annual_inc|verification_status|addr_state|zip_code|application_type|
+---+---------+----------+--------------+----------+-------------------+----------+--------+----------------+
|  0|   152606|    137908|             0|         0|                  0|         0|       1|               0|
+---+---------+----------+--------------+----------+-------------------+----------+--------+----------------+



## Investors:

- funded_amnt: المبلغ الذي وافقت عليه المنصة / تمت الموافقة على تمويله
- funded_amnt_inv: المبلغ الذي دفعه المستثمرون بالفعل (التمويل الحقيقي)
- out_prncp: المبلغ المتبقي الذي لم يسدده العميل من المبلغ الأصلي المقترض
    > لا يشمل الفوائد أو الرسوم، فقط أصل القرض
- out_prncp_inv: الرصيد المتبقي من الجزء الذي دفعه المستثمرون
- total_pymnt:  إجمالي المدفوعات على كامل القرض
- total_pymnt_inv: المدفوعات التي حصل عليها المستثمرون فقط
- recoveries: هو المبلغ المالي الذي تم استرداده من المقترض بعد أن تم اعتبار القرض خاسرًا
- collection_recovery_fee: هو المبلغ الذي تم خصمه كـ رسوم تحصيل بعد أن تم اعتبار القرض خاسرًا


In [7]:
investor_columns = ['funded_amnt', 'funded_amnt_inv', 'out_prncp', 'out_prncp_inv',
  'total_pymnt', 'total_pymnt_inv', 'collection_recovery_fee', 'recoveries']

investor = loans.select(investor_columns)
investor.printSchema()

root
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- out_prncp: string (nullable = true)
 |-- out_prncp_inv: string (nullable = true)
 |-- total_pymnt: string (nullable = true)
 |-- total_pymnt_inv: string (nullable = true)
 |-- collection_recovery_fee: string (nullable = true)
 |-- recoveries: string (nullable = true)



In [8]:
from pyspark.sql.functions import col, round

investor = investor \
    .withColumn("funded_amnt", col("funded_amnt").cast("int")) \
    .withColumn("funded_amnt_inv", col("funded_amnt_inv").cast("int")) \
    .withColumn("out_prncp", col("out_prncp").cast("float")) \
    .withColumn("out_prncp_inv", col("out_prncp_inv").cast("float")) \
    .withColumn("total_pymnt", round(col("total_pymnt").cast("float"), 2)) \
    .withColumn("total_pymnt_inv", col("total_pymnt_inv").cast("float")) \
    .withColumn("collection_recovery_fee", col("collection_recovery_fee").cast("int")) \
    .withColumn("recoveries", col("recoveries").cast("int")) 

investor.printSchema()

root
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: integer (nullable = true)
 |-- out_prncp: float (nullable = true)
 |-- out_prncp_inv: float (nullable = true)
 |-- total_pymnt: float (nullable = true)
 |-- total_pymnt_inv: float (nullable = true)
 |-- collection_recovery_fee: integer (nullable = true)
 |-- recoveries: integer (nullable = true)



In [9]:
investor.show(10)

+-----------+---------------+---------+-------------+-----------+---------------+-----------------------+----------+
|funded_amnt|funded_amnt_inv|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|collection_recovery_fee|recoveries|
+-----------+---------------+---------+-------------+-----------+---------------+-----------------------+----------+
|      12000|          12000|      0.0|          0.0|   13500.44|       13500.44|                      0|         0|
|      32000|          32000|  6158.89|      6158.89|   31838.67|       31838.67|                      0|         0|
|      40000|          40000|  22376.9|      22376.9|   30417.91|       30417.91|                      0|         0|
|      16000|          16000|  2455.43|      2455.43|   15527.57|       15527.57|                      0|         0|
|      33000|          33000|  9873.08|      9873.08|   26616.93|       26616.93|                      0|         0|
|       7000|           7000|   232.31|       232.31|    7588.07

In [10]:
investor.summary().toPandas()

25/07/26 04:54:51 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
                                                                                

,summary,funded_amnt,funded_amnt_inv,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,collection_recovery_fee,recoveries
0,count,2029952,2029952,2029951,2029952,2029952,2029952,2029952,2029952
1,mean,15194.058911245193,15189.156817993726,1819.912903476539,1819.6494393401342,14673.294574986183,14668.309777899456,38.740401250867016,220.6199757432688
2,stddev,9295.733305380894,9293.754523571126,4611.247911879345,4610.969606076708,10287.047518132387,10283.871818248797,170.62804175310157,956.531613264018
3,min,1000,725,0.0,0.0,0.0,0.0,0,0
4,25%,8000,8000,0.0,0.0,6789.81,6787.35,0,0
5,50%,13000,13000,0.0,0.0,11892.35,11888.75,0,0
6,75%,20000,20000,0.0,0.0,20136.63,20131.17,0,0
7,max,40000,40000,35165.32,35165.32,67174.38,67174.38,7174,39859


In [11]:
from pyspark.sql.functions import col, sum

investor_nulls = investor.select([sum(col(c).isNull().cast("int")).alias(c) for c in investor.columns])
investor_nulls.show()

[Stage 10:==========================================>              (9 + 3) / 12]

+-----------+---------------+---------+-------------+-----------+---------------+-----------------------+----------+
|funded_amnt|funded_amnt_inv|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|collection_recovery_fee|recoveries|
+-----------+---------------+---------+-------------+-----------+---------------+-----------------------+----------+
|          0|              0|        1|            0|          0|              0|                      0|         0|
+-----------+---------------+---------+-------------+-----------+---------------+-----------------------+----------+



In [12]:
investor.groupBy(col("out_prncp")).count().orderBy("count", ascending=True).show()

[Stage 13:============================>                            (6 + 6) / 12]

+---------+-----+
|out_prncp|count|
+---------+-----+
|  2783.83|    1|
|  9820.55|    1|
|  6967.29|    1|
|  6597.67|    1|
|  2464.55|    1|
| 21146.15|    1|
|  4676.69|    1|
|  6127.12|    1|
|  18508.6|    1|
| 13419.86|    1|
|  13038.6|    1|
| 11353.43|    1|
|  6381.72|    1|
|  8585.63|    1|
|  3382.57|    1|
| 20736.33|    1|
| 14320.76|    1|
| 11363.58|    1|
| 10445.01|    1|
|  4781.65|    1|
+---------+-----+
only showing top 20 rows



In [13]:
investor = investor.fillna({'out_prncp': 0.0})

In [14]:
investor_nulls = investor_nulls.select([sum(col(c).isNull().cast('int')).alias(c) for c in investor.columns])
investor_nulls.show()

[Stage 16:=========>                                              (2 + 10) / 12]

+-----------+---------------+---------+-------------+-----------+---------------+-----------------------+----------+
|funded_amnt|funded_amnt_inv|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|collection_recovery_fee|recoveries|
+-----------+---------------+---------+-------------+-----------+---------------+-----------------------+----------+
|          0|              0|        0|            0|          0|              0|                      0|         0|
+-----------+---------------+---------+-------------+-----------+---------------+-----------------------+----------+



## Credit History:
- earliest_cr_line >> earliest reported credit line was opened.
- > كلما كان تاريخ أول ائتمان أقدم، دلّ على خبرة ائتمانية أطول، وبالتالي مخاطرة أقل غالبًا.
- fico_range_low >> The lower boundary range (300-850). higher is good
- inq_last_6mths >> The number of inquiries in past 6 months -- lower is good
- > كلما زادت عدد الاستعلامات → زادت مخاطرة الائتمان (لأنه قد يدل على ضغط مالي).
- mths_since_last_delinq >> عدد الأشهر التي مرت منذ آخر حالة تخلف عن السداد في ملف المقترض الائتماني.
- mths_since_last_record >> The number of months since the last public record.
- > عدد الأشهر التي مرت منذ آخر سجل عام  على ملف المقترض الائتماني.
- open_acc >> The number of open credit lines in the borrower's credit file -- lower is better.
- total_acc >> The total number of credit lines currently in the borrower's credit file
- revol_util >> Revolving line utilization rate.
- > النسبة العالية (مثلاً 80%-90%) → مؤشر خطر: العميل بيستخدم نسبة كبيرة من رصيده.
- > النسبة المنخفضة (مثلاً 10%-30%) → مؤشر جيد: عنده تحكم في استخدامه للائتمان.
- delinq_2yrs >> The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years
- pub_rec >> Number of derogatory public records
- collections_12_mths_ex_med >> يمثل عدد حسابات التحصيل المسجلة ضد المقترض خلال آخر 12 شهر.



In [15]:
credit_history_columns = ['earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths',
     'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'total_acc',
     'revol_util', 'delinq_2yrs', 'pub_rec', 'collections_12_mths_ex_med']

credit_history = loans.select(credit_history_columns)
credit_history.limit(10).toPandas()

,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,total_acc,revol_util,delinq_2yrs,pub_rec,collections_12_mths_ex_med
0,Jun-1996,715.0,719.0,0.0,None,80.0,9.0,16.0,37%,0.0,1.0,0.0
1,Sep-2005,715.0,719.0,1.0,10.0,None,20.0,42.0,34.1%,2.0,0.0,0.0
2,Apr-2002,765.0,769.0,0.0,None,None,12.0,26.0,20.7%,0.0,0.0,0.0
3,Jun-2000,700.0,704.0,0.0,38.0,None,12.0,20.0,57.7%,0.0,0.0,0.0
4,Dec-2005,785.0,789.0,0.0,None,None,25.0,52.0,16.1%,0.0,0.0,0.0
5,Dec-2003,710.0,714.0,1.0,None,89.0,13.0,20.0,22.8%,0.0,1.0,0.0
6,Sep-2005,695.0,699.0,0.0,68.0,99.0,7.0,10.0,57.5%,0.0,1.0,0.0
7,Sep-1994,700.0,704.0,1.0,64.0,None,8.0,34.0,92.3%,0.0,0.0,0.0
8,Dec-1988,660.0,664.0,2.0,43.0,None,10.0,23.0,86%,0.0,0.0,0.0
9,Jan-2005,660.0,664.0,0.0,9.0,None,7.0,15.0,29.9%,1.0,0.0,0.0


In [16]:
from pyspark.sql.functions import col, round, regexp_replace, to_date

credit_history = credit_history \
    .withColumn("earliest_cr_line", to_date("earliest_cr_line", "MMM-yyyy")) \
    .withColumn("fico_range_low", col("fico_range_low").cast("int")) \
    .withColumn("fico_range_high", col("fico_range_high").cast("int")) \
    .withColumn("inq_last_6mths", col("inq_last_6mths").cast("int")) \
    .withColumn("mths_since_last_delinq", col("mths_since_last_delinq").cast("int")) \
    .withColumn("mths_since_last_record", col("mths_since_last_record").cast("int")) \
    .withColumn("open_acc", col("open_acc").cast("int")) \
    .withColumn("total_acc", col("total_acc").cast("int")) \
    .withColumn( "revol_util_ratio", regexp_replace(col("revol_util"), "%", "").cast("float") ) \
    .withColumn("delinq_2yrs", col("delinq_2yrs").cast("int")) \
    .withColumn("pub_rec", col("pub_rec").cast("int")) \
    .withColumn("collections_12_mths_ex_med", col("collections_12_mths_ex_med").cast("int")) 

credit_history.printSchema()

root
 |-- earliest_cr_line: date (nullable = true)
 |-- fico_range_low: integer (nullable = true)
 |-- fico_range_high: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- mths_since_last_delinq: integer (nullable = true)
 |-- mths_since_last_record: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- collections_12_mths_ex_med: integer (nullable = true)
 |-- revol_util_ratio: float (nullable = true)



In [17]:
credit_history.limit(10).toPandas()

,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,total_acc,revol_util,delinq_2yrs,pub_rec,collections_12_mths_ex_med,revol_util_ratio
0,1996-06-01,715,719,0,NaN,80.0,9,16,37%,0,1,0,37.000000
1,2005-09-01,715,719,1,10.0,NaN,20,42,34.1%,2,0,0,34.099998
2,2002-04-01,765,769,0,NaN,NaN,12,26,20.7%,0,0,0,20.700001
3,2000-06-01,700,704,0,38.0,NaN,12,20,57.7%,0,0,0,57.700001
4,2005-12-01,785,789,0,NaN,NaN,25,52,16.1%,0,0,0,16.100000
5,2003-12-01,710,714,1,NaN,89.0,13,20,22.8%,0,1,0,22.799999
6,2005-09-01,695,699,0,68.0,99.0,7,10,57.5%,0,1,0,57.500000
7,1994-09-01,700,704,1,64.0,NaN,8,34,92.3%,0,0,0,92.300003
8,1988-12-01,660,664,2,43.0,NaN,10,23,86%,0,0,0,86.000000
9,2005-01-01,660,664,0,9.0,NaN,7,15,29.9%,1,0,0,29.900000


In [18]:
from pyspark.sql.functions import col, sum

credit_history = credit_history.select([sum(col(c).isNull().cast("int")).alias(c) for c in credit_history.columns])
credit_history.limit(5).toPandas()

,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,total_acc,revol_util,delinq_2yrs,pub_rec,collections_12_mths_ex_med,revol_util_ratio
0,1,1,0,1,1024002,1691922,0,1,1587,0,0,0,1587


In [19]:
credit_history.summary().toPandas()

,summary,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,total_acc,revol_util,delinq_2yrs,pub_rec,collections_12_mths_ex_med,revol_util_ratio
0,count,1,1,1,1,1,1,1,1,1,1,1,1,1
1,mean,1.0,1.0,0.0,1.0,1024002.0,1691922.0,0.0,1.0,1587.0,0.0,0.0,0.0,1587.0
2,stddev,None,None,None,None,None,None,None,None,None,None,None,None,None
3,min,1,1,0,1,1024002,1691922,0,1,1587,0,0,0,1587
4,25%,1,1,0,1,1024002,1691922,0,1,1587,0,0,0,1587
5,50%,1,1,0,1,1024002,1691922,0,1,1587,0,0,0,1587
6,75%,1,1,0,1,1024002,1691922,0,1,1587,0,0,0,1587
7,max,1,1,0,1,1024002,1691922,0,1,1587,0,0,0,1587


In [20]:
credit_history.filter(col("revol_util_ratio")> 100).count()

1

## Fact loans
- dti (Debt-to-Income Ratio) >> Total Monthly Debt Payments / Monthly Income -- less than 36% is good.
- total_rec_prncp >> المبلغ المسدّد من أصل القرض
- total_rec_int >> إجمالي الفوائد المدفوعة من المقترض حتى الآن
- total_rec_late_fee >> هو إجمالي الرسوم المتأخرة التي تم دفعها من قبل المقترض حتى الآن بسبب تأخره عن السداد
- last_pymnt_amnt >> هو قيمة آخر دفعة قام المقترض بسدادها على هذا القرض

In [21]:
fact_columns = ['id', 'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade',
     'loan_status', 'pymnt_plan', 'purpose', 'dti', 'total_pymnt',
     'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
     'last_pymnt_amnt', 'fico_range_low', 'fico_range_high']

fact = loans.select(fact_columns)
fact.limit(5).toPandas()

,id,loan_amnt,term,int_rate,installment,grade,sub_grade,loan_status,pymnt_plan,purpose,dti,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,fico_range_low,fico_range_high
0,120122535,12000.0,36 months,7.97%,375.88,A,A5,Fully Paid,n,debt_consolidation,27.74,13500.44,12000.0,1500.44,0.0,2591.95,715.0,719.0
1,119374887,32000.0,36 months,11.99%,1062.71,B,B5,Current,n,credit_card,12.35,31838.67,25841.11,5997.56,0.0,0.0,715.0,719.0
2,119321612,40000.0,60 months,15.05%,952.65,C,C4,Current,n,debt_consolidation,31.11,30417.91,17623.1,12794.81,0.0,952.65,765.0,769.0
3,120122034,16000.0,36 months,7.97%,501.17,A,A5,Current,n,debt_consolidation,15.94,15527.57,13544.57,1983.0,0.0,0.0,700.0,704.0
4,118659541,33000.0,36 months,7.21%,1022.12,A,A3,Late (31-120 days),n,debt_consolidation,19.06,26616.93,23126.92,3490.01,0.0,0.0,785.0,789.0


In [22]:
from pyspark.sql.functions import col, regexp_replace, when

fact = fact \
    .withColumn("loan_amnt", col("loan_amnt").cast("int")) \
    .withColumn("term", regexp_replace("term", "months", "").cast("int")) \
    .withColumn("int_rate", regexp_replace("int_rate", "%", "").cast("float")) \
    .withColumn("installment", col("installment").cast("float")) \
    .withColumn("grade", col("grade").cast("string")) \
    .withColumn("sub_grade", col("sub_grade").cast("string")) \
    .withColumn("loan_status", col("loan_status").cast("string")) \
    .withColumn("pymnt_plan",
                when(col("pymnt_plan") == "n", "No")
                .when(col("pymnt_plan") == "n", "Charged Off")) \
    .withColumn("purpose", col("purpose").cast("string")) \
    .withColumn("dti", col("dti").cast("float")) \
    .withColumn("total_pymnt", col("total_pymnt").cast("float")) \
    .withColumn("total_rec_prncp", col("total_rec_prncp").cast("float")) \
    .withColumn("total_rec_int", col("total_rec_int").cast("float")) \
    .withColumn("total_rec_late_fee", col("total_rec_late_fee").cast("int")) \
    .withColumn("last_pymnt_amnt", col("last_pymnt_amnt").cast("float")) \
    .withColumn("fico_range_low", col("fico_range_low").cast("int")) \
    .withColumn("fico_range_high", col("fico_range_high").cast("int")) 

fact.limit(5).toPandas()  

,id,loan_amnt,term,int_rate,installment,grade,sub_grade,loan_status,pymnt_plan,purpose,dti,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,fico_range_low,fico_range_high
0,120122535,12000,36,7.97,375.880005,A,A5,Fully Paid,No,debt_consolidation,27.740000,13500.440430,12000.000000,1500.439941,0,2591.949951,715,719
1,119374887,32000,36,11.99,1062.709961,B,B5,Current,No,credit_card,12.350000,31838.669922,25841.109375,5997.560059,0,0.000000,715,719
2,119321612,40000,60,15.05,952.650024,C,C4,Current,No,debt_consolidation,31.110001,30417.910156,17623.099609,12794.809570,0,952.650024,765,769
3,120122034,16000,36,7.97,501.170013,A,A5,Current,No,debt_consolidation,15.940000,15527.570312,13544.570312,1983.000000,0,0.000000,700,704
4,118659541,33000,36,7.21,1022.119995,A,A3,Late (31-120 days),No,debt_consolidation,19.059999,26616.929688,23126.919922,3490.010010,0,0.000000,785,789


In [23]:
fact.printSchema()

root
 |-- id: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- term: integer (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- total_pymnt: float (nullable = true)
 |-- total_rec_prncp: float (nullable = true)
 |-- total_rec_int: float (nullable = true)
 |-- total_rec_late_fee: integer (nullable = true)
 |-- last_pymnt_amnt: float (nullable = true)
 |-- fico_range_low: integer (nullable = true)
 |-- fico_range_high: integer (nullable = true)



In [24]:
fact.groupBy("loan_status").count().show()

[Stage 32:==============================================>         (10 + 2) / 12]

+------------------+-------+
|       loan_status|  count|
+------------------+-------+
|        Fully Paid|1239089|
|           Default|    353|
|   In Grace Period|   5644|
|       Charged Off| 313663|
|Late (31-120 days)|   9532|
|           Current| 460148|
| Late (16-30 days)|   1522|
|        2015-10-01|      1|
+------------------+-------+



In [25]:
fact.groupBy("pymnt_plan").count().show()

[Stage 35:===================================================>    (11 + 1) / 12]

+----------+-------+
|pymnt_plan|  count|
+----------+-------+
|        No|2029951|
|      NULL|      1|
+----------+-------+



In [26]:
fact.groupBy("purpose").count().show(50)

[Stage 38:>                                                       (0 + 12) / 12]

+--------------------+-------+
|             purpose|  count|
+--------------------+-------+
|         educational|      2|
|               other| 126118|
|      small_business|  19952|
|  debt_consolidation|1146650|
|         credit_card| 468324|
|              moving|  13736|
|            vacation|  14216|
|    renewable_energy|   1217|
|               house|  12617|
|                 car|  20447|
|      major_purchase|  44475|
|             medical|  25216|
|    home_improvement| 136961|
|             wedding|     20|
|https://lendingcl...|      1|
+--------------------+-------+



In [27]:
fact.summary().toPandas()

,summary,id,loan_amnt,term,int_rate,installment,grade,sub_grade,loan_status,pymnt_plan,purpose,dti,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,fico_range_low,fico_range_high
0,count,2029952,2029952,2029952,2029952,2029952,2029952,2029952,2029952,2029951,2029952,2028240,2029952,2029952,2029952,2029849,2029951,2029951,2029952
1,mean,8.897918565635936E7,15194.07270467479,43.04202660949618,13.001447772632929,448.5704863708189,None,None,None,None,None,19.10218575232718,14673.294589981762,11629.925834085443,2820.43187058281,2.254738652973694,4061.7354656591856,698.4499502697356,702.4501613831263
2,stddev,3.896570662546707E7,9295.732593461107,10.92787988071443,4.872039042454465,269.8033562982502,None,None,None,None,None,14.722648903655609,10287.0475129867,8674.071455290406,2916.9254617533015,16.063623019494283,6477.518194399884,33.13907561111458,33.139988141878575
3,min,100001133,1000,36,5.31,7.61,A,A1,2015-10-01,No,car,-1.0,0.0,0.0,0.0,0,-2042.98,660,664
4,25%,5.9412448E7,8000,36,9.44,252.4,None,None,None,None,None,12.05,6789.8135,5000.0,890.99,0,295.51,675,679
5,50%,9.1615215E7,13000,36,12.61,378.42,None,None,None,None,None,18.05,11892.351,9619.16,1827.09,0,710.92,690,694
6,75%,1.25798585E8,20000,60,15.61,597.78,None,None,None,None,None,24.84,20136.625,16000.0,3726.21,0,5469.24,715,719
7,max,99997759,40000,60,30.99,1719.83,G,G5,Late (31-120 days),No,wedding,999.0,67174.38,40000.0,32589.8,1598,42192.05,845,850


In [28]:
from pyspark.sql.functions import col, sum

fact = fact.select([sum(col(c).isNull().cast("int")).alias(c) for c in fact.columns])
fact.limit(5).toPandas()

,id,loan_amnt,term,int_rate,installment,grade,sub_grade,loan_status,pymnt_plan,purpose,dti,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,fico_range_low,fico_range_high
0,0,0,0,0,0,0,0,0,1,0,1712,0,0,0,103,1,1,0


## Time
- issue_d >> The month which the loan was funded.
- last_pymnt_d >> Last month payment was received.
- > nulls:
 >  القرض جديد جدًا ولسه العميل مدفعش أول قسط.
 > أو العميل تأخر في الدفع أو تخلف تمامًا
    
- next_pymnt_d >> Next scheduled payment date
- > null >>  ده معناه غالبًا إن القرض تم إغلاقه أو تخلف العميل عن السداد وانتهى الجدول الزمني.
- last_credit_pull_d >> التاريخ اللي فيه تم سحب التقرير الائتماني  للمقترض آخر مرة

In [29]:
time_columns = ['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

time = loans.select(time_columns)
time.limit(5).toPandas()

,issue_d,last_pymnt_d,next_pymnt_d,last_credit_pull_d
0,2017-09-01,May-2020,None,May-2020
1,2017-09-01,May-2020,Jul-2020,May-2020
2,2017-09-01,May-2020,Jun-2020,May-2020
3,2017-09-01,May-2020,Jun-2020,May-2020
4,2017-09-01,Apr-2020,Jun-2020,May-2020


In [30]:
time.printSchema()

root
 |-- issue_d: string (nullable = true)
 |-- last_pymnt_d: string (nullable = true)
 |-- next_pymnt_d: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [31]:
from pyspark.sql.functions import to_date

time = time \
    .withColumn("issue_d", col("issue_d").cast("date")) \
    .withColumn("last_pymnt_d", to_date("last_pymnt_d", "MMM-yyyy")) \
    .withColumn("next_pymnt_d", to_date("next_pymnt_d", "MMM-yyyy")) \
    .withColumn("last_credit_pull_d", to_date("last_credit_pull_d", "MMM-yyyy")) 

time.limit(5).toPandas()

,issue_d,last_pymnt_d,next_pymnt_d,last_credit_pull_d
0,2017-09-01,2020-05-01,None,2020-05-01
1,2017-09-01,2020-05-01,2020-07-01,2020-05-01
2,2017-09-01,2020-05-01,2020-06-01,2020-05-01
3,2017-09-01,2020-05-01,2020-06-01,2020-05-01
4,2017-09-01,2020-04-01,2020-06-01,2020-05-01


In [32]:
time_nulls = time.select([sum(col(c).isNull().cast("int")).alias(c) for c in time.columns])
time_nulls.show()

[Stage 49:=================================>                       (7 + 5) / 12]

+-------+------------+------------+------------------+
|issue_d|last_pymnt_d|next_pymnt_d|last_credit_pull_d|
+-------+------------+------------+------------------+
|      1|        2191|     1552753|                59|
+-------+------------+------------+------------------+

